In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu-nodes"
pytcompute_target = ComputeTarget(workspace=ws, name=cluster_name)
print('Found existing compute target.')
print(compute_target.get_status().serialize())

In [ ]:
ds = ws.get_default_datastore()

In [ ]:
path_on_datastore = 'fowl_data'
ds_data = ds.path(path_on_datastore)
print(ds_data)

In [ ]:
import os

project_folder = './pytorch-birds'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
import shutil

shutil.copy('pytorch_train.py', project_folder)

In [ ]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_dir': ds_data.as_mount(),
    '--num_epochs': 30,
    '--output_dir': './outputs'
}

pytestimator = PyTorch(source_directory=project_folder, 
                    #script_params=script_params,
                    compute_target=pytcompute_target,
                    entry_script='pytorch_train.py',
                    use_gpu=True,
                    pip_packages=['pillow==5.4.1'])

# Create Pipeline using Estimator Step

In [ ]:
from azureml.core import Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData

input_data = DataReference(
    datastore=ds,
    data_reference_name="pytorch_fowl_data",
    path_on_datastore="fowl_data")

output = PipelineData("output", datastore=ds)

In [ ]:
from azureml.pipeline.steps import EstimatorStep

est_step = EstimatorStep(name="pytorch_train", 
                         estimator=pytestimator, 
                         estimator_entry_script_arguments=['--data_dir', input_data,'--num_epochs', 30, '--output_dir', './outputs'],
                         runconfig_pipeline_params=None, 
                         inputs=[input_data], 
                         outputs=[output], 
                         compute_target=pytcompute_target)

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment
pipeline = Pipeline(workspace=ws, steps=[est_step])
pipeline_run = Experiment(ws, 'pytorch-birds-pipeline').submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
publishedPipeline = pipeline_run.publish_pipeline(name="pytorch-birds-pipeline", description="sample pytorch pipeline",version="0.1")
print("Newly published pipeline id => ",publishedPipeline.id)
print("Newly published pipeline endpoint => ", publishedPipeline.endpoint)

# Resources

#### EstimatorStep Samples

https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-how-to-use-estimatorstep.ipynb
    
#### Invoke Published Pipeline REST API:
https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-publish-and-run-using-rest-endpoint.ipynb

#### Authentication Samples:
https://github.com/Azure/MachineLearningNotebooks/blob/1f05157d24c8bd9866121b588e75dc95764ae898/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb

#### Setup Service Principal

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.authentication.serviceprincipalauthentication?view=azure-ml-py

#### Calling REST API from Postman

##### OAuth 2.0 Access Token:
##### ------------------------------------------------

Grant Type = Client Credentials

Access Token Url = https: //login.microsoftonline.com/[tenant-id]/oauth2/token

ClientID = [App ID]

Client Secret = [Client Secret]

Scope = [Scopes URI under Expose an API]
    


